# Build pointing data

This notebook reads data produced by `reducing_pointing_data` notebook and analyses it to produce corrected pointing data files.

## Parameterized notebook

This notebook is parameterized, which means it could be run with tools like Papermill as part of a data analysis pipeline.

In [ ]:
import os 
import pickle

import numpy as np
import matplotlib.pyplot as plt

from astropy import units as u
from datetime import datetime

from lsst.geom import PointD

from lsst.ts.observing.utilities.auxtel.latiss.utils import calculate_xy_offsets

from lsst.ts.observatory.control.constants.latiss_constants import boresight

## Notebook Parameters

The next cell define the notebook parameters

### Pickle file with the data to process

The notebooks needs the name of the pickle file containing the data to process.

In additional users can provide values for the roundness rejection algorithm.
There are two levels of roundness rejection that can be applies, by value and by standard deviation.

Rejection by value will cause the algorithm to reject any data with roundess larger than the specified value.
Roundess is define as the ratio between largest to lowest moment, so it is always larger than 1.
If the user provides a value lower than 1 the notebook will raise a `RuntimeError` exception.

Rejection by standard deviation allow users to provide a multiplicative factor to the standard deviation to use as rejection level.

If both are provided, they are both applied to the dataset. 
Basically whatever is the more restrictive one will win.

For instance, assume a user provide `roundness_rejection_value = 1.25` and `roundness_rejection_std = 3.`, and the data contains a mean of `1.01` and standard deviation of `0.05`.
By standard deviation rejection all data larger than `1.01 + 3. * 0.05 = 1.16` will be rejected and the value rejection will not do any difference.
But if the data had a standard deviation of `0.09`, the standard deviation rejection level would be `1.01 + 3. * 0.09 = 1.28` and the rejection value would be the effective one.

By default both roundness rejections are `None` which means, do not apply any rejection.

In all cases, the data is still going to be written to the processed pointing file, but the rejected data will be marked with ";" so it gets rejected by `tpoint`.

In [ ]:
pointing_data_file = "data/20210609/AT_point_data_20210609_tw003.pickle"
roundness_rejection_value = None
roundness_rejection_std = None

In [ ]:
if roundness_rejection_value is not None and roundness_rejection_value < 1.:
    raise RuntimeError(f"Roundness rejection value must be larger than 1. Got {roundness_rejection_value}.")

## Processing the data

In [ ]:
with open(pointing_data_file, "rb") as fp:
    brightest_source_centroid = pickle.load(fp)

In [ ]:
def rotation_matrix(angle):
    """Rotation matrix.
    """
    return np.array(
        [
            [np.cos(np.radians(angle)), -np.sin(np.radians(angle)), 0.0],
            [np.sin(np.radians(angle)), np.cos(np.radians(angle)), 0.0],
            [0.0, 0.0, 1.0],
        ]
    )

In [ ]:
azel_correction = np.zeros((2, len(brightest_source_centroid)))

for i, source_xy in enumerate(brightest_source_centroid):
    dx_arcsec, dy_arcsec = calculate_xy_offsets(
        PointD(
            source_xy.brightestObjCentroid[0],
            source_xy.brightestObjCentroid[1]
        ), 
        boresight)

    # We are using rotator 2 so we must apply a negative sign on the x-axis offset.
    # The equation bellow return offset in elevation/azimuth.
    elaz_offset = np.matmul((-dx_arcsec, dy_arcsec, 0.), rotation_matrix(source_xy.angle))*u.arcsec
    
    # Note that the offsets bellow are the negative values of those applied
    # to the telescope to correct an image motion due to hexapod motion.
    # The reason is that a hexapod motion will be registered in the image 
    # as the negative of that applied to correct it. So basically I need to
    # "subtract" the image motion to the measured position.
    elaz_offset[0] -= source_xy.aos_offset["x"]*50.468*u.arcsec  # elevation
    elaz_offset[1] += source_xy.aos_offset["y"]*52.459*u.arcsec  # azimuth
    
    # We want to store the offset in azel format, so we reverse the result given above.
    # The following was verified with the pointing component. When we add an offset of 
    # X arcsec in azimuth it results in a negative offset in the axis. When we make a
    # positive offset in elevation is results in a positive offset in the axis. The 
    # pointing takes care of the cos(elevation) dependency when we apply the offset, but
    # we need to take care of it here since we want to apply a correction to the axis directly.    
    azel_correction[0][i] = elaz_offset[1].to(u.deg).value * -1. / np.cos(np.radians(source_xy.elevation))
    azel_correction[1][i] = elaz_offset[0].to(u.deg).value

In [ ]:
plt.scatter(azel_correction[0], azel_correction[1])

## Apply correction to pointing data

Now that the corrections offsets are computed in az/el, we need to apply the offset to the appropriate columns and construct the pointing data. 

In [ ]:
pointing_data = np.array(
    [
        tuple(
            [
                data.point_data.get(key,"") for key in ("mask", "expectedAzimuth", "expectedElevation", "measuredAzimuth", "measuredElevation", "measuredRotator")
            ]
        )
        for data in brightest_source_centroid
    ],
    dtype = [(key, float if key !=  "mask" else (np.unicode_, 1)) for key in ("mask", "expectedAzimuth", "expectedElevation", "measuredAzimuth", "measuredElevation", "measuredRotator")]
)

In [ ]:
corrected_pointing_data = pointing_data.copy()
corrected_pointing_data["measuredAzimuth"] += azel_correction[0]
corrected_pointing_data["measuredElevation"] += azel_correction[1]

In [ ]:
out_pointing_file, ext = os.path.splitext(pointing_data_file)
print(out_pointing_file, ext)

In [ ]:
now = datetime.now()

## Running rejection algorithm

In [ ]:
xXyY = np.array([source_xy.brightestObj_xXyY for source_xy in brightest_source_centroid]).T

In [ ]:
roundness = xXyY[0]/xXyY[1]

In [ ]:
roundess_mask = roundness < 1.

In [ ]:
roundness[roundess_mask] = 1./roundness[roundess_mask]

In [ ]:
roundness_mean = np.mean(roundness[np.isfinite(roundness)])
roundness_std = np.std(roundness[np.isfinite(roundness)])

In [ ]:
roudness_std_threshold = (roundness_mean + roundness_std*roundness_rejection_std) if roundness_rejection_std is not None else 0.

In [ ]:
roundess_value_threshold = roundness_rejection_value if roundness_rejection_value is not None else 0.

In [ ]:
roudness_threshold = roundess_value_threshold if roundess_value_threshold >= roudness_std_threshold else roudness_std_threshold

In [ ]:
good_data = np.isfinite(roundness)

if roudness_threshold > 0.:
    print(f"Applying roundness threshold cut: {roudness_threshold}")
    good_data = np.bitwise_and(good_data, roundness < roudness_threshold)

In [ ]:
plt.hist(roundness)

ylim = plt.ylim()

mean_roundness_plot = plt.plot([roundness_mean, roundness_mean], ylim, ":")

high_std_roundness_plot = plt.plot([roundness_mean+roundness_std, roundness_mean+roundness_std], ylim, ":", color=mean_roundness_plot[0].get_color())
high2_std_roundness_plot = plt.plot([roundness_mean+2*roundness_std, roundness_mean+2*roundness_std], ylim, ":", color=mean_roundness_plot[0].get_color())
high3_std_roundness_plot = plt.plot([roundness_mean+3*roundness_std, roundness_mean+3*roundness_std], ylim, ":", color=mean_roundness_plot[0].get_color())

if roudness_threshold > 0.:
    threshold_roundness_plot = plt.plot([roudness_threshold, roudness_threshold], ylim, "--")
plt.ylim(ylim)

In [ ]:
n_good_data = len(np.where(good_data)[0])
print(f"Masked data: {len(good_data) - n_good_data}")

In [ ]:
corrected_pointing_data["mask"][np.bitwise_not(good_data)] = ";"

## Saving data

In [ ]:
header = f"""LSST Auxiliary Telescope, {now.year} {now.month} {now.day} UTC {now.hour} {now.minute} {now.second}
: ALTAZ
: ROTNL
-30 14 40.3
"""
tail = "END"

In [ ]:
raw_filename = f"{out_pointing_file}.dat"
corrected_filename = f"{out_pointing_file}_corr.dat"

In [ ]:
with open(raw_filename, "w") as fp:
    print(f"Writting raw data to {raw_filename}.")
    fp.write(header)
    np.savetxt(fp, pointing_data, fmt="%s%011.7f %010.7f %011.7f %010.7f %011.7f")
    fp.write(tail)

with open(corrected_filename, "w") as fp:
    print(f"Writting corrected data to {corrected_filename}.")
    fp.write(header)
    np.savetxt(fp, corrected_pointing_data, fmt="%s%011.7f %010.7f %011.7f %010.7f %011.7f")
    fp.write(tail)